In [1]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geocoder
from geopy.geocoders import Nominatim


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
df1 = pd.read_csv('./data/01 EIDL through 111520.csv', error_bad_lines=False)
df2 = pd.read_csv('./data/02 EIDL through 111520.csv', error_bad_lines=False)
df3 = pd.read_csv('./data/03 EIDL through 111520.csv', error_bad_lines=False)
df4 = pd.read_csv('./data/04 EIDL through 111520.csv', error_bad_lines=False)
df5 = pd.read_csv('./data/05 EIDL through 111520.csv', error_bad_lines=False)

b'Skipping line 94905: expected 45 fields, saw 48\n'
b'Skipping line 571616: expected 45 fields, saw 46\n'
b'Skipping line 726662: expected 45 fields, saw 46\n'
b'Skipping line 766129: expected 45 fields, saw 46\n'
b'Skipping line 958568: expected 45 fields, saw 46\n'
/Users/songmei/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (13,18,23,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 302432: expected 45 fields, saw 46\n'
b'Skipping line 508518: expected 45 fields, saw 46\n'
b'Skipping line 955186: expected 45 fields, saw 46\n'
b'Skipping line 338270: expected 45 fields, saw 46\n'
b'Skipping line 500896: expected 45 fields, saw 46\n'
b'Skipping line 656910: expected 45 fields, saw 46\n'
b'Skipping line 768234: expected 45 fields, saw 46\n'
b'Skipping line 775762: expected 45 fields, saw 48\n'
b'Skipping line 839741: expe

In [4]:
smallb = pd.concat([df1,df2,df3,df4,df5])
smallb = smallb[['AWARDEEORRECIPIENTLEGALENTITYNAME','LEGALENTITYADDRLINE1','LEGALENTITYCITYNAME',
         'LEGALENTITYSTATECD','LEGALENTITYZIP5','LEGALENTITYZIPLAST4','LEGALENTITYCONGRESSIONALDISTRICT',
        'BUSINESSTYPES','FACEVALUEOFDIRECTLOANORLOANGUARANTEE','ORIGINALLOANSUBSIDYCOST']].copy()
smallb.dropna(inplace=True)
col_names = {'AWARDEEORRECIPIENTLEGALENTITYNAME':'business_name',
            'LEGALENTITYADDRLINE1':'street_address',
             'LEGALENTITYCITYNAME':'city',
             'LEGALENTITYSTATECD':'state',
            'LEGALENTITYZIP5':'zip_code_first5',
            'LEGALENTITYZIPLAST4':'zip_code_last4',
            'LEGALENTITYCONGRESSIONALDISTRICT':'congressional_district',
            'BUSINESSTYPES':'business_type',
            'FACEVALUEOFDIRECTLOANORLOANGUARANTEE':'loan_amount',
            'ORIGINALLOANSUBSIDYCOST':'loan_net_proceeds_UNSURE'}
smallb.rename(columns=col_names,inplace=True)

smallb.reset_index(inplace=True)

In [5]:
smallb.head(3)

,index,business_name,street_address,city,state,zip_code_first5,zip_code_last4,congressional_district,business_type,loan_amount,loan_net_proceeds_UNSURE
0,0,"Les R. Rager,DDS",1957 HOWELL MILL RD NW,ATLANTA,GA,30318,2522,5,PR,65300.0,8893.86
1,1,Bob L Rylaarsdam,647 Fairview Road,Grangeville,ID,83530,5184,1,PR,150000.0,20430.00
2,2,MYO Investments Inc,420 East Throckmorton Suite 200,Fort Worth,TX,76102,3755,12,R,150000.0,20430.00


In [6]:
smallb['addresses']=smallb['street_address'] + ', '+ smallb['city'] + ', ' + smallb['state']

In [7]:
smallb.head()

,index,business_name,street_address,city,state,zip_code_first5,zip_code_last4,congressional_district,business_type,loan_amount,loan_net_proceeds_UNSURE,addresses
0,0,"Les R. Rager,DDS",1957 HOWELL MILL RD NW,ATLANTA,GA,30318,2522,5,PR,65300.0,8893.86,"1957 HOWELL MILL RD NW, ATLANTA, GA"
1,1,Bob L Rylaarsdam,647 Fairview Road,Grangeville,ID,83530,5184,1,PR,150000.0,20430.00,"647 Fairview Road , Grangeville, ID"
2,2,MYO Investments Inc,420 East Throckmorton Suite 200,Fort Worth,TX,76102,3755,12,R,150000.0,20430.00,"420 East Throckmorton Suite 200, Fort Worth, TX"
3,3,P.O.E.T. Ind. Inc,114 12th St,Seal Beach,CA,90740,6429,48,R,48700.0,6632.94,"114 12th St, Seal Beach, CA"
4,4,Breuer Farms LLC,959 Ridge Avenue,Lancaster,WI,53813,2120,3,R,150000.0,20430.00,"959 Ridge Avenue, Lancaster, WI"


In [63]:
def title_maker(name):
    name = name.title().strip()
    return name
smallb['business_name'] = smallb['business_name'].map(title_maker)
smallb['street_address'] = smallb['street_address'].map(title_maker)
smallb['city'] = smallb['city'].map(title_maker)

In [64]:
smallb.head()

,index,business_name,street_address,city,state,zip_code_first5,zip_code_last4,congressional_district,business_type,loan_amount,loan_net_proceeds_UNSURE,addresses
0,0,"Les R. Rager,Dds",1957 Howell Mill Rd Nw,Atlanta,GA,30318,2522,5,PR,65300.0,8893.86,"1957 HOWELL MILL RD NW, ATLANTA, GA"
1,1,Bob L Rylaarsdam,647 Fairview Road,Grangeville,ID,83530,5184,1,PR,150000.0,20430.00,"647 Fairview Road , Grangeville, ID"
2,2,Myo Investments Inc,420 East Throckmorton Suite 200,Fort Worth,TX,76102,3755,12,R,150000.0,20430.00,"420 East Throckmorton Suite 200, Fort Worth, TX"
3,3,P.O.E.T. Ind. Inc,114 12Th St,Seal Beach,CA,90740,6429,48,R,48700.0,6632.94,"114 12th St, Seal Beach, CA"
4,4,Breuer Farms Llc,959 Ridge Avenue,Lancaster,WI,53813,2120,3,R,150000.0,20430.00,"959 Ridge Avenue, Lancaster, WI"


In [67]:
sbfinal = smallb[(smallb['city'] == 'Denver')&(smallb['state'] == 'CO')].copy()

In [68]:
sbfinal['state'].value_counts()

CO    10458
Name: state, dtype: int64

In [69]:
sbfinal.head()

,index,business_name,street_address,city,state,zip_code_first5,zip_code_last4,congressional_district,business_type,loan_amount,loan_net_proceeds_UNSURE,addresses
182,182,Thriftcon Llc,3799 N Williams St Unit,Denver,CO,80205,3491,1,R,25400.0,3459.48,"3799 N Williams St Unit, Denver, CO"
196,196,No More Locked Doors Ministries,1625 Xanthia Street,Denver,CO,80220,2219,1,MR,12900.0,1756.98,"1625 Xanthia Street, Denver, CO"
236,236,95 S. Cherokee Llc,95 S. Cherokee,Denver,CO,80223,1627,1,R,80000.0,10896.00,"95 S. Cherokee, denver, CO"
1136,1136,"Because We Lowry, Llc",101 Ulster Ct #104,Denver,CO,80230,7228,1,R,1000.0,136.20,"101 Ulster Ct #104, Denver, CO"
1497,1498,Big Daddy'S Bear Valley Llc,2772 S Wadsworth Blvd,Denver,CO,80227,3408,1,R,177300.0,24148.26,"2772 S WADSWORTH BLVD, DENVER, CO"


In [70]:
sbfinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10458 entries, 182 to 3761954
Data columns (total 12 columns):
index                       10458 non-null int64
business_name               10458 non-null object
street_address              10458 non-null object
city                        10458 non-null object
state                       10458 non-null object
zip_code_first5             10458 non-null int64
zip_code_last4              10458 non-null object
congressional_district      10458 non-null object
business_type               10458 non-null object
loan_amount                 10458 non-null float64
loan_net_proceeds_UNSURE    10458 non-null float64
addresses                   10458 non-null object
dtypes: float64(2), int64(2), object(8)
memory usage: 1.0+ MB


In [71]:
#geocode sulotion credit to Abdishakur on medium

locator = Nominatim(user_agent='myGeocoder')

from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
sbfinal['location'] = sbfinal['addresses'].apply(geocode)
#3 - create longitude, laatitude and altitude from location column (returns tuple)
sbfinal['point'] = sbfinal['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('1401 Lawrence Street Suite 1600 , Denver, CO',), **{}).
Traceback (most recent call last):
  File "/Users/songmei/opt/anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/Users/songmei/opt/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/Users/songmei/opt/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/songmei/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/Users/songmei/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 376, in _make_request
   

In [86]:
sbfinal['point'].dropna(inplace=True)

In [89]:
sbfinal.dropna(inplace=True)

In [92]:
sbfinal["point"].value_counts()

(39.7400213, -104.9747742, 0.0)                   50
(39.7615445, -104.9790183, 0.0)                   21
(39.7684127, -105.0252042, 0.0)                   17
(39.7495824, -104.9873761, 0.0)                   16
(39.727912950000004, -104.99979167532128, 0.0)    16
                                                  ..
(39.72063884353676, -104.95714548517209, 0.0)      1
(39.71620579507193, -104.92197054334251, 0.0)      1
(39.779766638072765, -104.7713641846071, 0.0)      1
(39.69088469445674, -104.98054567231837, 0.0)      1
(39.737404549999994, -104.95544477499999, 0.0)     1
Name: point, Length: 3906, dtype: int64

In [93]:
sbfinal['point'].drop_duplicates(inplace=True)

In [100]:
sbfinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4851 entries, 196 to 3758415
Data columns (total 14 columns):
index                       4851 non-null int64
business_name               4851 non-null object
street_address              4851 non-null object
city                        4851 non-null object
state                       4851 non-null object
zip_code_first5             4851 non-null int64
zip_code_last4              4851 non-null object
congressional_district      4851 non-null object
business_type               4851 non-null object
loan_amount                 4851 non-null float64
loan_net_proceeds_UNSURE    4851 non-null float64
addresses                   4851 non-null object
location                    4851 non-null object
point                       4851 non-null object
dtypes: float64(2), int64(2), object(10)
memory usage: 561.1+ KB


In [108]:
def zip_ranker(num):
    if num < 0.25:
        num = 'Village Friend: Making a Difference in this Zip'
        return num
    elif num >= 0.25 and num < 0.5:
        num = 'Village Benefactor: Your Making a Lasting Impact in this Zip'
        return num
    elif num >= 0.5 and num < 0.75:
        num = 'Village Patron: Your Impact is Going a Long Way in this Zip'
        return num
    elif num >= 0.75:
        num = 'Village Champ: Highest and Lasting Impact from Your Support in this Zip'
        return num
    else:
        num = 'Village Friend: Making a Difference in this Zip'
        return num
    
def type_ranker(num):
    if num < 0.25:
        num = 'Friend of the Business: Making a Difference for this Type'
        return num
    elif num >= 0.25 and num < 0.5:
        num = 'One of the Regulars: You\'re Making a Lasting Impact for this Type'
        return num
    elif num >= 0.5 and num < 0.75:
        num = 'Planting Roots: Your Impact is Going a Long Way for this Type'
        return num
    elif num >= 0.75:
        num = 'Business Partner: Highest and Lasting Impact from Your Support for this Type'
        return num
    else:
        num = 'Friend of the Business: Making a Difference for this Type'
        return num

def full_loan_ranker(num):
    if num < 0.25:
        num = 'Village Friend: Making a Big Difference'
        return num
    elif num >= 0.25 and num < 0.5:
        num = 'Village Benefactor: Your Making a Lasting Impact'
        return num
    elif num >= 0.5 and num < 0.75:
        num = 'Village Patron: Your Impact is Going a Long Way'
        return num
    elif num >= 0.75:
        num = 'Village Champ: Highest and Lasting Impact from Your Support'
        return num
    else:
        num = 'Village Friend: Making a Big Difference'
        return num
    
sbfinal['loan_size_rank_by_state'] = sbfinal['loan_amount'].rank(pct=True)
sbfinal['loan_amount_by_zip_percentile'] = sbfinal.groupby('zip_code_first5')[['loan_amount']].rank(pct=True)
sbfinal['loan_amount_by_type_percentile'] = sbfinal.groupby('business_type')[['loan_amount']].rank(pct=True)
sbfinal['loan_amount_by_city_percentile'] = sbfinal.groupby('city')[['loan_amount']].rank(pct=True)
sbfinal['zipcode_urgency'] = sbfinal['loan_amount_by_zip_percentile'].apply(zip_ranker)
sbfinal['type_urgency'] = sbfinal['loan_amount_by_type_percentile'].apply(type_ranker)
sbfinal['loan_size_urgency'] = sbfinal['loan_size_rank_by_state'].apply(full_loan_ranker)

In [109]:
sbfinal.head()

,index,business_name,street_address,city,state,zip_code_first5,zip_code_last4,congressional_district,business_type,loan_amount,loan_net_proceeds_UNSURE,addresses,location,point,loan_size_rank_by_state,loan_amount_by_zip_percentile,loan_amount_by_type_percentile,loan_amount_by_city_percentile,zipcode_urgency,type_urgency,loan_size_urgency
196,196,No More Locked Doors Ministries,1625 Xanthia Street,Denver,CO,80220,2219,1,MR,12900.0,1756.98,"1625 Xanthia Street, Denver, CO","(1625, Xanthia Street, Denver, Denver County, ...","(39.74248495923504, -104.88710013250387, 0.0)",0.292826,0.299373,0.242424,0.292826,Village Benefactor: Your Making a Lasting Impa...,Friend of the Business: Making a Difference fo...,Village Benefactor: Your Making a Lasting Impact
1497,1498,Big Daddy'S Bear Valley Llc,2772 S Wadsworth Blvd,Denver,CO,80227,3408,1,R,177300.0,24148.26,"2772 S WADSWORTH BLVD, DENVER, CO","(2772, South Wadsworth Boulevard, Denver, Denv...","(39.666190020408166, -105.08134720408164, 0.0)",0.988868,1.000000,0.983818,0.988868,Village Champ: Highest and Lasting Impact from...,Business Partner: Highest and Lasting Impact f...,Village Champ: Highest and Lasting Impact from...
1894,1895,Choice City Home Improvement Inc.,1324 S University Blvd,Denver,CO,80210,2405,1,R,53000.0,7218.60,"1324 S UNIVERSITY BLVD, DENVER, CO","(1324, South University Boulevard, Denver, Den...","(39.69240675, -104.95896760096582, 0.0)",0.643579,0.663793,0.538807,0.643579,Village Patron: Your Impact is Going a Long Wa...,Planting Roots: Your Impact is Going a Long Wa...,Village Patron: Your Impact is Going a Long Way
3309,3311,Rachel Barber,1669 Syracuse St.,Denver,CO,80220,2014,1,PR,1000.0,136.20,"1669 Syracuse St., Denver, CO","(1669, Syracuse Street, Denver, Denver County,...","(39.74319116326531, -104.898864, 0.0)",0.037518,0.029781,0.051700,0.037518,Village Friend: Making a Difference in this Zip,Friend of the Business: Making a Difference fo...,Village Friend: Making a Big Difference
3622,3624,5280 House Cleaning Inc,1417 Gaylord St,Denver,CO,80206,2110,1,R,500000.0,68100.00,"1417 Gaylord St, Denver, CO","(1417, Gaylord Street, Cheesman Park, Denver, ...","(39.7387887, -104.96136656379986, 0.0)",0.997732,1.000000,0.996704,0.997732,Village Champ: Highest and Lasting Impact from...,Business Partner: Highest and Lasting Impact f...,Village Champ: Highest and Lasting Impact from...


In [111]:
sbfinal.to_json('./data/denver.json',orient='records')

In [112]:
smallb.to_csv('./data/smallb.csv')